In [1]:
import lingam
import numpy as np
import pandas as pd
# import sys
from lingam.utils import make_dot
from scipy.stats import ks_2samp
from sklearn.linear_model import LinearRegression


# np.set_printoptions(precision=3, suppress=True, threshold=sys.maxsize)
np.random.seed(100)

# loading the dataset
high_scrap = pd.read_csv("dataset/high_scrap.csv")
low_scrap = pd.read_csv("dataset/low_scrap.csv")


scale_data = lambda X: (X - X.min()) / (X.max() - X.min())

high_scrap = scale_data(high_scrap)
low_scrap = scale_data(low_scrap)

full = pd.concat([high_scrap, low_scrap])
# including previous knowledge as a matrix
# prior knowledge matrix that have as many rows and columns as the number of columns in the dataset with values -1

def prior_knowledge_matrix(columns):
    """
    prior knowledge matrix for LiNGAM where:
    0: no directed path possible (temporal constraint violation)
    1: directed path 
    -1: no prior knowledge (we'll allow the algorithm to determine)
    """
    n_features = len(columns)
    prior_knowledge = np.full((n_features, n_features), -1)
    
    # get station number 
    def get_station_number(col_name):
        return int(col_name.split('_')[0].replace('Station', ''))
    
    # get measurement point number
    def get_mp_number(col_name):
        return int(col_name.split('_')[2])
    
    for i in range(n_features):
        for j in range(n_features):
            station_i = get_station_number(columns[i])
            station_j = get_station_number(columns[j])
            
            # constraint
            if station_i > station_j:
                prior_knowledge[i, j] = 0
            
            # should we allow internal dependencies? 
            # if station_i == station_j:
            #     prior_knowledge[i, j] = -1  # Let LiNGAM determine
    
    # No self loop allowed
    np.fill_diagonal(prior_knowledge, 0)
    
    return prior_knowledge


# Create the prior knowledge matrix
prior_knowledge = prior_knowledge_matrix(full.columns)

prior_knowledge

array([[ 0, -1, -1, ..., -1, -1, -1],
       [-1,  0, -1, ..., -1, -1, -1],
       [-1, -1,  0, ..., -1, -1, -1],
       ...,
       [ 0,  0,  0, ...,  0, -1, -1],
       [ 0,  0,  0, ..., -1,  0, -1],
       [ 0,  0,  0, ..., -1, -1,  0]])

In [2]:
from causallearn.search.FCMBased import lingam
model = lingam.DirectLiNGAM(11, prior_knowledge)
model.fit(full)

In [3]:
fitted_adj = pd.DataFrame(model.adjacency_matrix_)
fitted_adj.columns = low_scrap.columns
fitted_adj.index = low_scrap.columns
fitted_adj

,Station1_mp_0,Station1_mp_1,Station1_mp_2,Station1_mp_3,Station1_mp_4,Station1_mp_5,Station2_mp_6,Station2_mp_7,Station2_mp_8,Station2_mp_9,...,Station5_mp_88,Station5_mp_89,Station5_mp_90,Station5_mp_91,Station5_mp_92,Station5_mp_93,Station5_mp_94,Station5_mp_95,Station5_mp_96,Station5_mp_97
Station1_mp_0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.509769,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
Station1_mp_1,-0.025527,0.000000,0.209999,0.0,0.0,0.000000,0.0,0.0,-0.979713,0.0,...,0.0,0.0,0.00000,-0.011348,0.0,0.0,0.000000,0.0,0.0,0.0
Station1_mp_2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
Station1_mp_3,0.000000,0.000000,0.000000,0.0,0.0,0.044563,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
Station1_mp_4,0.000000,0.084519,0.000000,0.0,0.0,0.249915,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Station5_mp_93,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
Station5_mp_94,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
Station5_mp_95,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.34991,-0.620201,0.0,0.0,0.000000,0.0,0.0,0.0
Station5_mp_96,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [4]:
import networkx as nx

causal_graph = nx.from_pandas_adjacency(fitted_adj, create_using=nx.DiGraph)


In [5]:
from dowhy import gcm


causal_model = gcm.ProbabilisticCausalModel(causal_graph)  # (X, Y) -> Z -> W
gcm.auto.assign_causal_mechanisms(causal_model, low_scrap)

In [6]:
attributions_robust = gcm.distribution_change(causal_model, low_scrap, high_scrap, 'Station5_mp_85', num_samples=100)
attributions_robust

Estimating Shapley Values. Average change of Shapley values in run 59 (295 evaluated permutations): 42.37006754627205%:   0%|          | 0/1 [51:00<?, ?it/s] 

In [ ]:
attributions_robust

NameError: name 'attributions_robust' is not defined